# Born Rule

This notebook contains the programmatic verification for the **Born Rule** entry from the THEORIA dataset.

**Entry ID:** born_rule  
**Required Library:** sympy 1.12.0

## Description
The Born rule specifies how a quantum state yields measurable probabilities for any well-defined measurement. Given a state and a measurement description, it assigns a normalized probability to each possible outcome in a way consistent with observed frequencies under repeated trials. This rule provides the operational meaning of the quantum state and sets the predictive, testable character of quantum theory across experiments and applications.

## Installation
First, let's install the required library:

In [ ]:
# Install required library with exact version
!pip install sympy==1.12.0

## Programmatic Verification

The following code verifies the derivation mathematically:

In [ ]:
import sympy as sp

# ===============================================
# Programmatic verification for Born rule derivation
# Following Steps 1–10 in the derivation
# ===============================================

# We work in a 3-dimensional Hilbert space (dim(H) = 3), as required by Gleason conditions.
# Use a standard orthonormal basis e1, e2, e3 represented as column vectors.
d = 3
I = sp.eye(d)

# -------------------------------------------------------------------
# Step 1–2: Probability measure on rank-1 projectors and frame function
# -------------------------------------------------------------------

# Basis vectors
e1 = sp.Matrix([1, 0, 0])
e2 = sp.Matrix([0, 1, 0])
e3 = sp.Matrix([0, 0, 1])

# Rank-1 projectors P_(e_i) = |e_i)(e_i|
P_e1 = e1 * e1.T
P_e2 = e2 * e2.T
P_e3 = e3 * e3.T

# Define a generic diagonal density matrix rho with nonnegative eigenvalues p1, p2, p3
p1, p2, p3 = sp.symbols('p1 p2 p3', nonnegative=True)
rho = sp.diag(p1, p2, p3)

# Impose trace-one condition Tr(rho) = 1 (this encodes normalization of the probability measure)
trace_rho = sp.trace(rho)

# Frame function on basis vectors: f(e_i) = mu(P_(e_i)) = Tr(rho P_(e_i))
f_e1 = sp.trace(rho * P_e1)
f_e2 = sp.trace(rho * P_e2)
f_e3 = sp.trace(rho * P_e3)

# Step 1 / 2 check: sum_i f(e_i) = Tr(rho) and equals 1 when rho is normalized
assert sp.simplify(f_e1 + f_e2 + f_e3 - trace_rho) == 0

# -------------------------------------------------------------------
# Steps 3–5: Define q(v), verify parallelogram identity and polarization
# -------------------------------------------------------------------

# Define generic complex components of a vector v and w
v1, v2, v3 = sp.symbols('v1 v2 v3', complex=True)
w1, w2, w3 = sp.symbols('w1 w2 w3', complex=True)

v = sp.Matrix([v1, v2, v3])
w = sp.Matrix([w1, w2, w3])

# For the purposes of programmatic verification, we *define*
# q(v) = <v|rho|v> = v^† rho v, which is quadratic in v.
# In the derivation, this is what is *derived* from the assumptions;
# here we check that once q has this form, all the stated identities hold.
def q(vec):
    return (vec.conjugate().T * rho * vec)[0]

# Step 4: Check parallelogram identity q(v+w) + q(v-w) = 2 q(v) + 2 q(w)
lhs_par = sp.simplify(q(v + w) + q(v - w))
rhs_par = sp.simplify(2*q(v) + 2*q(w))
assert sp.simplify(lhs_par - rhs_par) == 0

# Step 5: Define the sesquilinear form B via polarization and check q(v) = B(v,v)
def B(vec1, vec2):
    # Full polarization identity for complex Hilbert spaces
    return sp.simplify((q(vec1 + vec2) - q(vec1 - vec2) + sp.I*q(vec1 + sp.I*vec2) - sp.I*q(vec1 - sp.I*vec2)) / 4)

Bvv = sp.simplify(B(v, v))
assert sp.simplify(Bvv - q(v)) == 0

# -------------------------------------------------------------------
# Step 6: Riesz representation consistency check
# -------------------------------------------------------------------

# Here, B(v,w) should equal <w|rho|v> (note the swapped order due to sesquilinearity). We check this explicitly.
Bvw = sp.simplify(B(v, w))
inner_form = (w.conjugate().T * rho * v)[0]
assert sp.simplify(Bvw - inner_form) == 0

# This shows that the operator playing the role of T in the derivation
# is represented here by rho.

# -------------------------------------------------------------------
# Step 7: Positivity and trace-one of rho
# -------------------------------------------------------------------

# rho is diagonal with nonnegative entries by construction, so <v|rho|v> >= 0
# For symbolic verification, we check that q(e_i) equals the diagonal entries
# and thus is nonnegative if p_i >= 0.

q_e1 = sp.simplify(q(e1))
q_e2 = sp.simplify(q(e2))
q_e3 = sp.simplify(q(e3))

assert sp.simplify(q_e1 - p1) == 0
assert sp.simplify(q_e2 - p2) == 0
assert sp.simplify(q_e3 - p3) == 0

# Trace condition Tr(rho) = p1 + p2 + p3
assert sp.simplify(trace_rho - (p1 + p2 + p3)) == 0

# -------------------------------------------------------------------
# Step 8: mu(P) = Tr(rho P) for arbitrary projectors built from basis
# -------------------------------------------------------------------

# Consider an arbitrary projector onto the span of {e1, e2}:
# P_12 = |e1)(e1| + |e2)(e2|
P_12 = P_e1 + P_e2

# The measure mu(P_12) should be mu(P_e1) + mu(P_e2) by sigma-additivity,
# and also equal Tr(rho P_12).
mu_P12_from_additivity = f_e1 + f_e2
mu_P12_from_trace = sp.trace(rho * P_12)

assert sp.simplify(mu_P12_from_additivity - mu_P12_from_trace) == 0

# -------------------------------------------------------------------
# Step 9: Pure state case and amplitude-squared Born rule
# -------------------------------------------------------------------

# Let |psi> be a generic normalized vector in the 3D Hilbert space.
a1, a2, a3 = sp.symbols('a1 a2 a3', complex=True)
psi = sp.Matrix([a1, a2, a3])

# Pure state density operator rho_psi = |psi)(psi|
rho_psi = psi * psi.conjugate().T

# Probability of outcome corresponding to P_e1 is:
P_e1_prob = sp.simplify(sp.trace(rho_psi * P_e1))
# This should be |a1|^2
P_e1_expected = sp.simplify(a1 * sp.conjugate(a1))
assert sp.simplify(P_e1_prob - P_e1_expected) == 0

# Similarly for P_e2 and P_e3
P_e2_prob = sp.simplify(sp.trace(rho_psi * P_e2))
P_e3_prob = sp.simplify(sp.trace(rho_psi * P_e3))

P_e2_expected = sp.simplify(a2 * sp.conjugate(a2))
P_e3_expected = sp.simplify(a3 * sp.conjugate(a3))

assert sp.simplify(P_e2_prob - P_e2_expected) == 0
assert sp.simplify(P_e3_prob - P_e3_expected) == 0

# -------------------------------------------------------------------
# Step 10: POVM consistency in the projective case
# -------------------------------------------------------------------

# For a simple check of Step 10, we treat a PVM {Pi_i} also as a POVM {E_i}
# with E_i = Pi_i. This corresponds to the special case where the POVM arises
# directly as a projective measurement without dilation.

# Restrict to a 2D subspace spanned by e1, e2 and define a 2x2 density matrix.
r00, r01, r10, r11 = sp.symbols('r00 r01 r10 r11', complex=True)
rho_2 = sp.Matrix([[r00, r01], [r10, r11]])

# Define projectors P0 = |e1><e1|, P1 = |e2><e2| in this subspace:
P0 = sp.Matrix([[1, 0], [0, 0]])
P1 = sp.Matrix([[0, 0], [0, 1]])

# As a POVM, take E0 = P0, E1 = P1. The Born rule should give identical probabilities
# whether we view them as PVM elements or POVM effects.
P0_prob_pvm = sp.trace(rho_2 * P0)
P1_prob_pvm = sp.trace(rho_2 * P1)

P0_prob_povm = sp.trace(rho_2 * P0)  # E0 = P0
P1_prob_povm = sp.trace(rho_2 * P1)  # E1 = P1

assert sp.simplify(P0_prob_pvm - P0_prob_povm) == 0
assert sp.simplify(P1_prob_pvm - P1_prob_povm) == 0

print("Born rule derivation checks passed (Steps 1–10).")


## Source

📖 **View this entry:** [theoria-dataset.org/entries.html?entry=born_rule.json](https://theoria-dataset.org/entries.html?entry=born_rule.json)

This verification code is part of the [THEORIA dataset](https://github.com/theoria-dataset/theoria-dataset), a curated collection of theoretical physics derivations with programmatic verification.

**License:** CC-BY 4.0